# RT2 Assignment

- Create a Jupyter Notebook to replace User Interface (Node A)
- Use widgets to let the user know the position of the robot and all targets that have been set and cancelled in the environment & the distance of the closest obstacle

## Notebook

Import the Required Libraries:

In [1]:
import jupyros as jr
import rospy
import sys
import select
import time
import math
import actionlib
import actionlib.msg
import assignment_2_2022
import assignment_2_2022.msg
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import RobotMsg
from std_srvs.srv import *
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation

%matplotlib widget

Initialize the Node:

In [ ]:
rospy.init_node('jupyter_notebook')
pub = rospy.Publisher("/pos_vel", RobotMsg, queue_size = 1)
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
client.wait_for_server()

Implement the widgets:

In [8]:
# Buttons for Selecting Modes

send_goal_button = widgets.Button(value = False, description = "Target Position",
                                  disabled = False, button_style = '')

cancel_goal_button = widgets.Button(value = False, description = "Cancel Position", 
                                    disabled = False, button_style = '')

In [9]:
# Numeric Widgets
x = widgets.BoundedFloatText(value = 0.0, description = 'x', min = 9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))
y = widgets.BoundedFloatText(value = 0.0, description = 'y', min = 9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))

In [12]:
def callback_send(msg):
    global cancelled
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    client.send_goal(goal)

In [13]:
def callback_cancel(msg):
    global cancelled 
    client.cancel_goal()

In [ ]:
send_goal_button.on_click(callback_send)
cancel_goal_button.on_click(callback_cancel)

In [ ]:
def odom_callback(msg):
    
    

In [ ]:
def laser_callback(msg):
    

In [ ]:
# Position Visualization

class Odom_Visualizer:
    def __init__(self):
        self.fig_odom, self.ax = plt.subplots()
        # Robot Position Plot
        self.ln, = plt.plot([], [], 'ro', label = 'Robot Position')
        # Target's Position Plot
        self.goal_ln, = plt.plot([], [], 'b*', markersize = 10, label = 'Goal Position')
        # Robot's Position Data Arrays
        self.x_data, self.y_data = [], []
        
    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Robot Odometry", fontsize = 20, fontweight = 'bold')
        # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize = 10, fontweight = "bold")
        self.ax.set_ylabel("Y [m]", fontsize = 10, fontweight = "bold")
        # Set Plot Axis Limits
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        # Set Grid to True
        self.ax.grid(True)
        return self.ln, self.goal_ln
    
    def odom_callback(self, msg):
        # Callback Function used to update data
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)         
        
    def update_plot(self, frame):
        # Update Robot's Position on plot
        self.ln.set_data(self.x_data, self.y_data)        
        return self.ln, self_goal_ln

In [ ]:
# Goal Visualizer

class Goal_Visualizer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        plt.grid(False)
        # Setting up the values
        self.reached = 0
        self.cancelled = 0
        
    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Goals are reached or not", fontsize = 20, fontweight = "bold")
        # Set Plot Labels on x-axis
        self.labels('Reached', 'Cancelled')
        self.x_pos = np.arrange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        # Set Plot Axis Limits
        self.ax.set_ylim([0, 10])
        self.ax.set_yticks(np.arange(0, 11, 1))
        self.ln = plt.bar(['Reached', 'Unreached'], [self.reached, self.unreached], color = ['green', 'red'])
        return self.ln
    
    def goal_callback(self, msg):
        # Get the number of reached/cancelled goals
        self.reached = msg.reached
        self.cancelled = msg.cancelled     
        
    def update_plot(self, frame):
        self.ln = plt.bar(['Reached', 'Cancelled'], [self.reached, self.unreached], color = ['green', 'red'])
        return self.ln

In [ ]:
display(widgets.HBox([x, y]))
display(widgets.HBox([send_goal_button, cancel_goal_button]))

In [ ]:
current_pos = 0
jr.subscribe('/odom', Odometry, callback_odom)

In [ ]:
laser = 0
jr.subscribe('/scan', LaserScan, callback_laser)

In [ ]:
visualize_goals = Goal_Visualizer()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, visualize_goals.goal_callback)

animation_goal = FuncAnimation(visualize_goals.fig, visualize_goals.update_plot, interval = 1000)
plt.show(block = True)

In [ ]:
psition_visualizer = Odom_Visualizer()
sub = rospy.Subscriber('/odom', Odometry, position_visualizer.odom_callback)

animation_pos = FuncAnimation(position_visualizer.fig, position_visualizer.update_plot, init_func = position_visualizer.plot_init)
plt.show(block = True)